# Prescriptive analytics

## Using additive aggregation in Python

In [53]:
from io import StringIO
import pandas as pd
import numpy as np

### Load and prepare the decision matrix and the alternatives

In [57]:
data = StringIO("""Alternative;Production_cost;Production_volume;Expected_lifetime;Production_time;Return_rate;price
1) Direct marketing of a quality product;20;20000;5;8;0,005;50
2) Branding by utility companies;22;50000;5;10;0,005;80
3) Branding by shower installation companies;25;30000;5;12;0,005;100
4) Partnering with smart home vendors;30;2000;10;16;0,001;120
5) Creating a cheap functioning product;15;100000;3;12;0,01;40

""")
DM = pd.read_csv(data, sep=";", decimal=",")
alternative = DM['Alternative']
DM = DM.drop('Alternative', axis=1)
#print(DM)
print(DM.dtypes)
#for column_name in DM.rows:



Production_cost        int64
Production_volume      int64
Expected_lifetime      int64
Production_time        int64
Return_rate          float64
price                  int64
dtype: object


### Normalize the criteria values 

In [58]:
DM_normalized = DM / DM.sum()

#Alternatively, using the apply() method
#DM_normalized = DM.apply(lambda x: x / x.sum(), axis=0)

print(DM_normalized)

   Production_cost  Production_volume  Expected_lifetime  Production_time  \
0         0.178571           0.099010           0.178571         0.137931   
1         0.196429           0.247525           0.178571         0.172414   
2         0.223214           0.148515           0.178571         0.206897   
3         0.267857           0.009901           0.357143         0.275862   
4         0.133929           0.495050           0.107143         0.206897   

   Return_rate     price  
0     0.192308  0.128205  
1     0.192308  0.205128  
2     0.192308  0.256410  
3     0.038462  0.307692  
4     0.384615  0.102564  


### Compute the utility per alternative

In [59]:
utilities = pd.Series(DM_normalized.sum(axis=1), name="utility")

U = pd.concat([alternative, utilities], axis=1)
print(U)

                                    Alternative   utility
0      1) Direct marketing of a quality product  0.914597
1              2) Branding by utility companies  1.192374
2  3) Branding by shower installation companies  1.205915
3         4) Partnering with smart home vendors  1.256917
4       5) Creating a cheap functioning product  1.430197


### Obtain the optimal alternative

In [60]:
max_utility_index = U['utility'].idxmax()
max_utility_row = U.loc[max_utility_index]
print("The optimal alternative is:")
print(U.loc[max_utility_index])

The optimal alternative is:
Alternative    5) Creating a cheap functioning product
utility                                       1.430197
Name: 4, dtype: object


## Linear programming

In [62]:
from scipy.optimize import linprog

The solver interface from scipy: You put the values from the system above into the appropriate lists, tuples, or NumPy arrays:

| Variable | Content |
|----------|---------|
| obj | holds the coefficients from the objective function.|
| lhs_ineq | holds the left-side coefficients from the inequality (red, blue, and yellow) constraints.|
| rhs_ineq | holds the right-side coefficients from the inequality (red, blue, and yellow) constraints.|
| lhs_eq | holds the left-side coefficients from the equality (green) constraint.|
| rhs_eq | holds the right-side coefficients from the equality (green) constraint.|

Note: Please, be careful with the order of rows and columns!

The order of the rows for the left and right sides of the constraints must be the same. Each row represents one constraint.

The order of the coefficients from the objective function and left sides of the constraints must match. Each column corresponds to a single decision variable.


In [67]:
obj = [-1, -2]
#      ─┬  ─┬
#       │   └┤ Coefficient for y
#       └────┤ Coefficient for x

lhs_ineq = [[ 2,  1],  # Red constraint left side
            [-4,  5],  # Blue constraint left side
            [ 1, -2]]  # Yellow constraint left side

rhs_ineq = [20,  # Red constraint right side
            10,  # Blue constraint right side
             2]  # Yellow constraint right side

lhs_eq = [[-1, 5]]  # Green constraint left side
rhs_eq = [15]       # Green constraint right side
bnd = [(0, float("inf")),  # Bounds of x
       (0, float("inf"))]  # Bounds of y

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
              method="highs")
opt

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -16.818181818181817
              x: [ 7.727e+00  4.545e+00]
            nit: 0
          lower:  residual: [ 7.727e+00  4.545e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00]
                 marginals: [-2.727e-01]
        ineqlin:  residual: [ 0.000e+00  1.818e+01  3.364e+00]
                 marginals: [-6.364e-01 -0.000e+00 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

### Exercise: Straton Case Company
**Objective Function**: In scipy.optimize.linprog, the problem is formulated as a minimization problem by default. Therefore, to maximize 34x1 + 40x2, we minimize -34x1 - 40x2. This is why the coefficients of the objective function c are negated.

**Constraints**: The constraints are specified in the form Ax <= b, similar to the R code.

**Bounds**: The bounds for the variables are specified to ensure they are non-negative.

**Solver**: The linprog function is called with the method set to 'highs', which is recommended for large problems and is generally more efficient.

In [69]:
#Coefficients of the objective function
c = [-34, -40]  # Multiply by -1 to convert max to min

# Coefficients of the inequality constraints
A = np.array([
    [4, 6],
    [2, 2],
    [2, 1]
])

# Right-hand side of inequality constraints
b = np.array([48, 18, 16])

# Bounds for variables (x1 >= 0, x2 >= 0)
x_bounds = [(0, None), (0, None)]

# Solve the linear program
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Print the results
print("Optimal value:", -result.fun)  # Multiply by -1 to get the original max value
print("Solution:", result.x)

Optimal value: 342.0
Solution: [3. 6.]
